In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input

# --- Configuration & Paths ---
DATA_PATH = 'data/copper_historical.csv'
OUTPUT_PATH = 'output/forecast_2026.csv'
WINDOW_SIZE = 24
EPOCHS = 50

# Ensure directories exist
os.makedirs('output', exist_ok=True)

# --- Data Loading & Preprocessing ---
def load_and_preprocess(path):
    df = pd.read_csv(path)
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
    df = df.sort_values('Date')
    prices = df[['Price']].values
    
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(prices)
    return df, scaled_data, scaler

def create_sequences(data, window):
    X, y = [], []
    for i in range(len(data) - window):
        X.append(data[i:i + window])
        y.append(data[i + window])
    return np.array(X), np.array(y)

# --- Model Architecture ---
def build_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(64, return_sequences=True),
        Dropout(0.2), # Prevents overfitting
        LSTM(32),
        Dropout(0.1),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# --- Main Execution ---
df, scaled_data, scaler = load_and_preprocess(DATA_PATH)
X_train, y_train = create_sequences(scaled_data, WINDOW_SIZE)

model = build_model((WINDOW_SIZE, 1))
print("Training Model...")
model.fit(X_train, y_train, epochs=EPOCHS, batch_size=32, verbose=1)

# --- Recursive Forecasting for 2026 ---
print("Generating 2026 Forecast...")
current_batch = scaled_data[-WINDOW_SIZE:].reshape(1, WINDOW_SIZE, 1)
future_predictions = []

for _ in range(365):
    pred = model.predict(current_batch, verbose=0)[0]
    future_predictions.append(pred)
    # Update window: shift and append new prediction
    current_batch = np.append(current_batch[:, 1:, :], [[pred]], axis=1)

# --- Results Preservation ---
forecast_prices = scaler.inverse_transform(future_predictions)
dates_2026 = pd.date_range(start='2026-01-01', periods=365)

forecast_df = pd.DataFrame({'Date': dates_2026, 'Predicted_Price': forecast_prices.flatten()})
forecast_df.to_csv(OUTPUT_PATH, index=False)
print(f"Results saved to {OUTPUT_PATH}")

In [ ]:
# --- Configuration & Paths ---
DATA_PATH = 'data/copper_historical.csv'
OUTPUT_PATH = 'output/forecast_2026.csv'
WINDOW_SIZE = 24
EPOCHS = 50

# Ensure directories exist
os.makedirs('output', exist_ok=True)

In [ ]:
# --- Data Loading & Preprocessing ---
def load_and_preprocess(path):
    df = pd.read_csv(path)
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
    df = df.sort_values('Date')
    prices = df[['Price']].values
    
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(prices)
    return df, scaled_data, scaler

In [ ]:

def create_sequences(data, window):
    X, y = [], []
    for i in range(len(data) - window):
        X.append(data[i:i + window])
        y.append(data[i + window])
    return np.array(X), np.array(y)

In [ ]:
# --- Model Architecture ---
def build_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(64, return_sequences=True),
        Dropout(0.2), # Prevents overfitting
        LSTM(32),
        Dropout(0.1),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model


In [ ]:
df, scaled_data, scaler = load_and_preprocess(DATA_PATH)
X_train, y_train = create_sequences(scaled_data, WINDOW_SIZE)

model = build_model((WINDOW_SIZE, 1))
print("Training Model...")
model.fit(X_train, y_train, epochs=EPOCHS, batch_size=32, verbose=1)

In [ ]:
# --- Recursive Forecasting for 2026 ---
print("Generating 2026 Forecast...")
current_batch = scaled_data[-WINDOW_SIZE:].reshape(1, WINDOW_SIZE, 1)
future_predictions = []


In [ ]:
for _ in range(365):
    pred = model.predict(current_batch, verbose=0)[0]
    future_predictions.append(pred)
    # Update window: shift and append new prediction
    current_batch = np.append(current_batch[:, 1:, :], [[pred]], axis=1)

# --- Results Preservation ---
forecast_prices = scaler.inverse_transform(future_predictions)
dates_2026 = pd.date_range(start='2026-01-01', periods=365)

forecast_df = pd.DataFrame({'Date': dates_2026, 'Predicted_Price': forecast_prices.flatten()})
forecast_df.to_csv(OUTPUT_PATH, index=False)
print(f"Results saved to {OUTPUT_PATH}")